In [1]:
from datetime import datetime
from modelhub import ModelHub
from bach import display_sql_as_markdown
import bach
import pandas as pd
# instantiate the model hub and set the default time aggregation to daily
modelhub = ModelHub(time_aggregation='%Y-%m-%d')

In [2]:
import os
# DB_BQ_CREDENTIALS_PATH = os.path.dirname(os.path.dirname(os.path.realpath('.secrets/'))) + ''
# df = modelhub.get_objectiv_dataframe(db_url='bigquery://objectiv-production/snowplow',
#                                      start_date='2022-05-10', end_date='2022-06-25',table_name='events',
#                                      bq_credentials_path=DB_BQ_CREDENTIALS_PATH)

df = modelhub.get_objectiv_dataframe(db_url='bigquery://objectiv-production/snowplow',
#                                      start_date='2022-05-10', end_date='2022-06-25',
                                     start_date='2022-05-25', end_date='2022-05-25',
                                     table_name='events',
                                bq_credentials_path='/home/vard/objectiv-production--bigquery-read-only.json')

In [3]:
# adding specific contexts to the data as columns
df['application'] = df.global_contexts.gc.application
df['feature_nice_name'] = df.location_stack.ls.nice_name
df['root_location'] = df.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')
# df['referrer'] = df.global_contexts.gc.get_from_context_with_type_series(type='HttpContext', key='referrer')
# df['utm_source'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='source')
# df['utm_medium'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='medium')
# df['utm_campaign'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='campaign')
# df['utm_content'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='content')
# df['utm_term'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='term')

In [4]:
bq_df = df.copy()

In [5]:
# get the Bach DataFrame with Dialog data > REPLACE WITH YOUR PG CREDENTIALS
df = modelhub.get_objectiv_dataframe(db_url='',
#                                      start_date='2022-05-10', end_date='2022-06-25',
                                     start_date='2022-05-25', end_date='2022-05-25',
                                     table_name='data')

In [6]:
# adding specific contexts to the data as columns
df['application'] = df.global_contexts.gc.application
df['feature_nice_name'] = df.location_stack.ls.nice_name
df['root_location'] = df.location_stack.ls.get_from_context_with_type_series(type='RootLocationContext', key='id')
# df['referrer'] = df.global_contexts.gc.get_from_context_with_type_series(type='HttpContext', key='referrer')
# df['utm_source'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='source')
# df['utm_medium'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='medium')
# df['utm_campaign'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='campaign')
# df['utm_content'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='content')
# df['utm_term'] = df.global_contexts.gc.get_from_context_with_type_series(type='MarketingContext', key='term')

In [7]:
pdf = df.to_pandas()
bq_pdf = bq_df.to_pandas()

# align data sets

In [8]:
merged_df = bq_pdf.merge(pdf,how='outer',left_index=True,right_index=True,suffixes=('_bq',''))

merged_df['in_both'] = False
merged_df.loc[pd.notnull(merged_df.user_id_bq) & pd.notnull(merged_df.user_id),'in_both'] = True

merged_df.loc[merged_df.moment != merged_df.moment_bq, 'in_both'] = False

bq_df['in_both'] = merged_df[~merged_df.in_both].in_both
df['in_both'] = merged_df[~merged_df.in_both].in_both

df = df[df.in_both.isnull()]
bq_df = bq_df[bq_df.in_both.isnull()]

In [9]:
print(df.user_id.count().value)
print(bq_df.user_id.count().value)

632
632


In [10]:
set(df['moment'].to_pandas().to_list()) - set(bq_df['moment'].to_pandas().to_list())

set()

In [11]:
set(bq_df['moment'].to_pandas().to_list()) - set(df['moment'].to_pandas().to_list()) 

set()

### Entries with wrong session_id

In [12]:
pp = df.to_pandas()[['moment', 'session_id', 'user_id']]
bb = bq_df.to_pandas()[['moment', 'session_id', 'user_id']]

In [13]:
pp = pp[(pp.moment > datetime(2022, 5, 25, 8)) & (pp.moment < datetime(2022, 5, 25, 10))]
bb = bb[(bb.moment > datetime(2022, 5, 25, 8)) & (bb.moment < datetime(2022, 5, 25, 10))]

In [14]:
pp.head()

,moment,session_id,user_id
event_id,,,
8c655808-4c3e-45a2-9209-34a3924fed89,2022-05-25 09:24:31.345,16,087d5ac5-f456-4422-a4f9-a4c0962f156b
5f42dbdc-637a-41ad-aebc-de7d9ae8e8c3,2022-05-25 08:20:17.832,10,22b3b826-291e-4916-b702-60b4e624ba29
5985db55-2669-4fdd-9a87-e9824176c667,2022-05-25 09:34:27.467,18,562da5cc-8f91-450a-8ee4-8d756709d653
fcf0f31f-3a61-4810-8726-7eca8fe51152,2022-05-25 09:34:28.094,18,562da5cc-8f91-450a-8ee4-8d756709d653
b5fb078c-778e-44f9-b561-89736e22c174,2022-05-25 09:05:52.244,14,57019d3f-aacb-4f88-aa24-e05959e94a49


In [15]:
bb.head()

,moment,session_id,user_id
event_id,,,
8c655808-4c3e-45a2-9209-34a3924fed89,2022-05-25 09:24:31.345,67,087d5ac5-f456-4422-a4f9-a4c0962f156b
5f42dbdc-637a-41ad-aebc-de7d9ae8e8c3,2022-05-25 08:20:17.832,42,22b3b826-291e-4916-b702-60b4e624ba29
5985db55-2669-4fdd-9a87-e9824176c667,2022-05-25 09:34:27.467,71,562da5cc-8f91-450a-8ee4-8d756709d653
fcf0f31f-3a61-4810-8726-7eca8fe51152,2022-05-25 09:34:28.094,71,562da5cc-8f91-450a-8ee4-8d756709d653
b5fb078c-778e-44f9-b561-89736e22c174,2022-05-25 09:05:52.244,55,57019d3f-aacb-4f88-aa24-e05959e94a49


In [16]:
from uuid import UUID
pp[pp['user_id'] == UUID('b4b5ce02-7215-4193-9417-2df2faae4b03')]['session_id'].value_counts()

12    56
6     37
Name: session_id, dtype: int64

In [17]:
bb[bb['user_id'] == UUID('b4b5ce02-7215-4193-9417-2df2faae4b03')]['session_id'].value_counts()

46    53
21    37
78     3
Name: session_id, dtype: int64

In [18]:
bb[bb['session_id'] == 78].head()

,moment,session_id,user_id
event_id,,,
f17173ce-ef95-4857-aec8-88ebe03bbf4f,2022-05-25 09:39:03.509,78,b4b5ce02-7215-4193-9417-2df2faae4b03
cbcf5f59-ad1f-4209-8b18-4688c948832f,2022-05-25 09:39:05.164,78,b4b5ce02-7215-4193-9417-2df2faae4b03
d6f8d5c1-dd5a-4251-9c7b-df5741145b44,2022-05-25 09:39:05.377,78,b4b5ce02-7215-4193-9417-2df2faae4b03


In [19]:
pp[pp['session_id'] == 12].head(100)

,moment,session_id,user_id
event_id,,,
54e9c427-5af0-402c-8dc7-aa5bd7af3d1e,2022-05-25 08:50:19.792,12,b4b5ce02-7215-4193-9417-2df2faae4b03
0f9d3b39-c7cd-424c-b732-01ac247d665c,2022-05-25 08:50:19.793,12,b4b5ce02-7215-4193-9417-2df2faae4b03
e7778a80-effa-40d0-ac1a-aeb7e82d1412,2022-05-25 08:50:19.794,12,b4b5ce02-7215-4193-9417-2df2faae4b03
f2234083-198c-446c-9339-17cbd71273d3,2022-05-25 08:50:19.795,12,b4b5ce02-7215-4193-9417-2df2faae4b03
a399901d-0f0b-41ae-8b00-c396a10e7347,2022-05-25 08:50:19.808,12,b4b5ce02-7215-4193-9417-2df2faae4b03
f4c8f4a9-ff8f-4896-9025-2a7427ad6a6a,2022-05-25 08:52:12.622,12,b4b5ce02-7215-4193-9417-2df2faae4b03
2b22923f-4210-428e-9e37-af1f61cabb22,2022-05-25 08:52:12.624,12,b4b5ce02-7215-4193-9417-2df2faae4b03
6e0d82c2-ea81-4fb3-ba32-70dfa101eba2,2022-05-25 08:52:12.624,12,b4b5ce02-7215-4193-9417-2df2faae4b03
23750cf4-8078-4edd-8f37-5f91d219d62a,2022-05-25 08:52:12.642,12,b4b5ce02-7215-4193-9417-2df2faae4b03


In [20]:
bb[bb['session_id'] == 46].head(100)

,moment,session_id,user_id
event_id,,,
54e9c427-5af0-402c-8dc7-aa5bd7af3d1e,2022-05-25 08:50:19.792,46,b4b5ce02-7215-4193-9417-2df2faae4b03
0f9d3b39-c7cd-424c-b732-01ac247d665c,2022-05-25 08:50:19.793,46,b4b5ce02-7215-4193-9417-2df2faae4b03
e7778a80-effa-40d0-ac1a-aeb7e82d1412,2022-05-25 08:50:19.794,46,b4b5ce02-7215-4193-9417-2df2faae4b03
f2234083-198c-446c-9339-17cbd71273d3,2022-05-25 08:50:19.795,46,b4b5ce02-7215-4193-9417-2df2faae4b03
a399901d-0f0b-41ae-8b00-c396a10e7347,2022-05-25 08:50:19.808,46,b4b5ce02-7215-4193-9417-2df2faae4b03
f4c8f4a9-ff8f-4896-9025-2a7427ad6a6a,2022-05-25 08:52:12.622,46,b4b5ce02-7215-4193-9417-2df2faae4b03
2b22923f-4210-428e-9e37-af1f61cabb22,2022-05-25 08:52:12.624,46,b4b5ce02-7215-4193-9417-2df2faae4b03
6e0d82c2-ea81-4fb3-ba32-70dfa101eba2,2022-05-25 08:52:12.624,46,b4b5ce02-7215-4193-9417-2df2faae4b03
23750cf4-8078-4edd-8f37-5f91d219d62a,2022-05-25 08:52:12.642,46,b4b5ce02-7215-4193-9417-2df2faae4b03


In [21]:
set(pp[pp['session_id'] == 12].moment) - set(bb[bb['session_id'] == 46].moment)

{Timestamp('2022-05-25 09:39:03.509000'),
 Timestamp('2022-05-25 09:39:05.164000'),
 Timestamp('2022-05-25 09:39:05.377000')}

In [ ]:
# postgres session_id == 12 has 3 wrong entries:
# Timestamp('2022-05-25 09:39:03.509000')
# Timestamp('2022-05-25 09:39:05.164000')
# Timestamp('2022-05-25 09:39:05.377000')